In [156]:
import pandas as pd
import numpy as np
from pathlib import Path
import requests
import os
import matplotlib.pyplot as plt
import urllib.request
import seaborn as sns
from platform import platform
from glob import iglob
import math
#supress warnings
import warnings
warnings.filterwarnings('ignore')
from numpy import median
from scipy.stats import norm
import re

import plotly
from plotly.offline import iplot

import plotly.graph_objects as go
# from wordcloud import WordCloud
import emoji
import string
import re
from nltk.corpus import stopwords

import string
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack
import pandas as pd
import os
import sys
sys.path.insert(0, '/Price_regression_using_CBIR/Retrieval')
import DeepImageUtils as IU


sns.set(style="darkgrid", font_scale=1.2)

ModuleNotFoundError: No module named 'DeepImageUtils'

In [132]:
current_folder = globals()['_dh'][0]
test_path = current_folder + '/test_records.csv'
validation_path = current_folder + '/validation_records.csv'
train_path = current_folder + '/train_records.csv'
test_df = pd.read_csv(Path(test_path), error_bad_lines=False)
validation_df = pd.read_csv(Path(validation_path), error_bad_lines=False)
train_df = pd.read_csv(Path(train_path), error_bad_lines=False)
#For this we merge training and valiadtion
train_df = pd.concat([train_df, validation_df])

train_df = train_df.drop('Unnamed: 0', 1)
train_df = train_df.drop('Unnamed: 0.1', 1)
test_df = test_df.drop('Unnamed: 0', 1)
test_df = test_df.drop('Unnamed: 0.1', 1)

In [22]:
print(train_df.iloc[1280])

uniq_id                                1b04ee78c86c7ac1f9df869a91dd5747
brand                                                          Fabindia
care_instructions                             Viscose&nbsp; | Hand-wash
dominant_material                                               Viscose
title                 Fabindia Rust Red & Black Handblock Print Butt...
actual_color                                         Red | Black | Rust
dominant_color                                                     Rust
product_type                                                      Shrug
images                http://assets.myntassets.com/v1/assets/images/...
product_details       Rust red and black printed longline shrug with...
complete_the_look     A fantastic cover-up for when the temperature ...
price                                                              2590
gender                                                            Women
specifications        Occasion : Ethnic | Main Trend : Ethnic Pr

In [ ]:
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text.decode('utf8'))

In [ ]:
def remove_punchuations(text):
    return text.translate(str.maketrans('', '', string.punctuation)).strip()

In [ ]:
def remove_contract_words(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stopwords.words('english')]

In [ ]:
def process_text(data, cols):
    for col in cols:

        processed_data = []

        for sentence in data[col].values:
            sent = remove_contract_words(sentence)
            # sent = sentence
            sent = re.sub("\s+", " ", sent)
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            sent = give_emoji_free_text(sent)
            sent = remove_punchuations(sent)
            sent = remove_stopwords(sent)
            processed_data.append(sent.lower().strip())

        data[col] = processed_data

    return data

In [3]:
category_path = globals()['_dh'][0] + '/total_features.npz'
loaded_features = np.load(category_path, allow_pickle=True)

In [7]:
features = list(loaded_features['arr_0'])

In [12]:
features[0][2]
len(features)

86042

In [10]:
uniq_ids = [feature[2] for feature in features]

In [11]:
len(uniq_ids)

86042

In [13]:
from collections import Counter

In [24]:
coun = Counter(uniq_ids)

In [36]:
res = list(coun.keys())

In [37]:
res[0]

'463018956bd93929f483b8a179f9e688'

In [72]:
# feature_dict = {key: None for key in keys}
train_df = pd.read_csv(globals()['_dh'][0] + "/train_records.csv")
validation_df = pd.read_csv(globals()['_dh'][0] + "/validation_records.csv")

In [88]:
feature_dict = dict()
for feature in features:
    if feature[2] in feature_dict:
#         if feature[1] == feature_dict[feature[2]]:
#             continue
        feature_dict[feature[2]].append(feature[1])
    else:
        feature_dict[feature[2]] = [feature[1]]

463018956bd93929f483b8a179f9e688
here
4792


ValueError: iAt based indexing can only have integer indexers

In [76]:
train_df.iloc[0]

Unnamed: 0                                                            0
Unnamed: 0.1                                                       7601
uniq_id                                e87417d6278a1d04085694d5b46a50d4
brand                                                             Libas
care_instructions                                     RayonMachine-wash
dominant_material                                                 Rayon
title                 Libas Women Mustard Yellow Hem Design Wide Leg...
actual_color                                           Yellow | Mustard
dominant_color                                                  Mustard
product_type                                           Wide Leg Palazzo
images                http://assets.myntassets.com/v1/assets/images/...
product_details       A pair of mustard yellow solid woven wide leg ...
complete_the_look     Show off your fun side this weekend with these...
price                                                           

In [65]:
feature_dict['463018956bd93929f483b8a179f9e688'][1] == feature_dict['463018956bd93929f483b8a179f9e688'][10]

IndexError: list index out of range

In [90]:
X_train = pd.read_csv(globals()['_dh'][0] + "/train_records.csv")
feature_dict = dict()
for feature in features:
        feature_dict[feature[2]] = [feature[1]]
train_features_in_order = []
for index, row in X_train.iterrows():
        train_features_in_order.insert(index, feature_dict[row.uniq_id])

In [91]:
X_train.iloc[0]

Unnamed: 0                                                            0
Unnamed: 0.1                                                       7601
uniq_id                                e87417d6278a1d04085694d5b46a50d4
brand                                                             Libas
care_instructions                                     RayonMachine-wash
dominant_material                                                 Rayon
title                 Libas Women Mustard Yellow Hem Design Wide Leg...
actual_color                                           Yellow | Mustard
dominant_color                                                  Mustard
product_type                                           Wide Leg Palazzo
images                http://assets.myntassets.com/v1/assets/images/...
product_details       A pair of mustard yellow solid woven wide leg ...
complete_the_look     Show off your fun side this weekend with these...
price                                                           

In [92]:
train_features_in_order[0]

[array([0.01092354, 0.01591033, 0.        , ..., 0.027866  , 0.00717207,
        0.0076995 ], dtype=float32)]

In [117]:
[len(a) for a in feature_dict['e87417d6278a1d04085694d5b46a50d4'][0]]

TypeError: object of type 'numpy.float32' has no len()

Testing metadata processing

In [118]:
def give_emoji_free_text(text):
    # return emoji.get_emoji_regexp().sub(r'', text)
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         "]+", flags=re.UNICODE)

    return pattern.sub(r'', text)

In [119]:
def remove_punchuations(sentence):
    # return text.translate(str.maketrans('', '', string.punctuation)).strip()
    regular_punct = list(string.punctuation)

    for punc in regular_punct:
        if punc in sentence:
            sentence = sentence.replace(punc, ' ')

    return sentence.strip()

In [120]:

def remove_contract_words(text):
    try:
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can\'t", "can not", text)
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'t", " not", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'m", " am", text)
    except Exception as exp:
        print(exp)
    return text

In [121]:

STOPWORDS = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [122]:
def remove_stopwords(text):
    # return ' '.join([word for word in text if word not in stopwords.words('english')])
    return ' '.join(e for e in text.split() if e not in STOPWORDS)

In [123]:
def process_text(data, cols):
    # print(data.columns.values.tolist())
    for col in cols:

        processed_data = []

        for sentence in data[col].values:
            sent = remove_contract_words(sentence)
            # sent = sentence
            try:
                sent = give_emoji_free_text(sent)
                sent = remove_punchuations(sent)
                sent = remove_stopwords(sent)
                sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
                sent = re.sub("\s+", " ", sent)
                sent = sent.lower().strip()
            except Exception as exp:
                print(exp)
            # if col == 'title':
            #     print(sent)
            processed_data.append(sent)

        data[col] = processed_data

    return data

In [124]:
# one hot encoding of category names
def get_vohe(X_train, X_validation, X_test, col_name):
    """
    Get one hot encoded features
    """
    vect = CountVectorizer()
    tr_ohe = vect.fit_transform(X_train[col_name].values)
    vl_ohe = vect.transform(X_validation[col_name].values)
    te_ohe = vect.transform(X_test[col_name].values)

    return tr_ohe, vl_ohe, te_ohe

In [126]:
def get_vtext_encodings(X_train, X_validation, X_test, col_name, min_val, max_val):
    """
    Get TFIDF encodings with max_features capped at 1M
    """
    vect = TfidfVectorizer(min_df=10, ngram_range=(min_val, max_val), max_features=1000000)
    tr_text = vect.fit_transform(X_train[col_name].values)
    vl_text = vect.transform(X_validation[col_name].values)
    te_text = vect.transform(X_test[col_name].values)

    return tr_text, vl_text, te_text

In [127]:
def get_test_and_validation_image_features(X_train, X_validation):
    loaded_features = np.load(image_features_path, allow_pickle=True)
    features = list(loaded_features['arr_0'])
    feature_dict = dict()
    for feature in features:
        feature_dict[feature[2]] = [feature[1]]
    train_features_in_order = []
    validation_features_in_order = []
    for index, row in X_train.iterrows():
        try:
            train_features_in_order.insert(index, feature_dict[row.uniq_id])
        except Exception as exp:
            print(exp)
            img = images_path + row['uniq_id'] + '_0.jpg'
            feature_vector = IU.CreateImageFeatureVector(img)
            train_features_in_order.insert(index, 
    for index, row in X_validation.iterrows():
        validation_features_in_order.insert(index, feature_dict[row.uniq_id])
    return train_features_in_order, validation_features_in_order

In [128]:
# def add_features_to_testdf(test_df):
#     test_features_in_order = []
#     for index, row in test_df.iterrows():
#         img = images_path + row['uniq_id'] + '_0.jpg'
#         # print(img)
#         feature_vector = IU.CreateImageFeaturesVector(img)
#         test_features_in_order.insert(index, feature_vector)
#     return test_features_in_order

In [145]:
def generate_encodings_for_features(X_train, X_validation, X_test):
    """
        Get encodings for all the features. Scale and normalize the numerical features. Stack the encoded features horizontally.
    """
    X_train = X_train.fillna('others')
    X_test = X_test.fillna('others')
    X_validation = X_validation.fillna('others')
    X_train = process_text(X_train,
                           ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    X_validation = process_text(X_validation,
                                ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    X_test = process_text(X_test,
                          ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    tr_ohe_brand, vl_ohe_brand, te_ohe_brand = get_vohe(X_train, X_validation, X_test, 'brand')
    # tr_ohe_dominant_material, te_ohe_dominant_material = get_vohe(X_train, X_test, 'dominant_material')
    tr_ohe_dominant_color, vl_ohe_dominant_color, te_ohe_dominant_color = get_vohe(X_train, X_validation, X_test, 'dominant_color')
    tr_ohe_product_type, vl_ohe_product_type, te_ohe_product_type = get_vohe(X_train, X_validation, X_test, 'product_type')
    tr_ohe_gender, vl_ohe_gender, te_ohe_gender = get_vohe(X_train, X_validation, X_test, 'gender')
    tr_ohe_category_1, vl_ohe_category_1, te_ohe_category_1 = get_vohe(X_train, X_validation, X_test, 'category_1')
    tr_ohe_category_0, vl_ohe_category_0, te_ohe_category_0 = get_vohe(X_train, X_validation, X_test, 'category_0')

    tr_trans = csr_matrix(
        pd.get_dummies(X_train[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)
    vl_trans = csr_matrix(
        pd.get_dummies(X_validation[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)
    te_trans = csr_matrix(
        pd.get_dummies(X_test[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)

    tr_title, vl_title, te_title = get_vtext_encodings(X_train, X_validation, X_test, 'title', 1, 1)
    tr_product_details, vl_product_details, te_product_details = get_vtext_encodings(X_train, X_validation, X_test, 'product_details', 1, 2)
    tr_complete_the_look, vl_complete_the_look, te_complete_the_look = get_vtext_encodings(X_train, X_validation, X_test, 'complete_the_look', 1, 2)
    tr_specifications, vl_specifications, te_specifications = get_vtext_encodings(X_train, X_validation, X_test, 'specifications', 1, 2)
    tr_ohe_dominant_material, vl_ohe_dominant_material, te_ohe_dominant_material = get_vtext_encodings(X_train, X_validation, X_test, 'dominant_material', 1, 1)
    tr_ohe_actual_color, vl_ohe_actual_color, te_ohe_actual_color = get_vtext_encodings(X_train, X_validation, X_test, 'actual_color', 1, 1)
    tr_ohe_care_instruction_1, vl_ohe_care_instruction_1, te_ohe_care_instruction_1 = get_vtext_encodings(X_train, X_validation, X_test, 'care_instruction_1', 1, 1)
    tr_ohe_care_instruction_0, vl_ohe_care_instruction_0, te_ohe_care_instruction_0 = get_vtext_encodings(X_train, X_validation, X_test, 'care_instruction_0', 1, 1)


    # Add image feature vectors also to hstack
    train_features_in_order, validation_features_in_order = get_test_and_validation_image_features(X_train, X_validation)
#     test_features_in_order = add_features_to_testdf(X_test)

#     train_data = hstack((tr_ohe_category_0, tr_ohe_category_1, tr_ohe_brand, tr_ohe_dominant_material,
#                          tr_ohe_dominant_color, tr_ohe_actual_color, tr_ohe_product_type, tr_trans, tr_ohe_gender,
#                          tr_ohe_care_instruction_0, tr_title, tr_product_details, tr_ohe_care_instruction_1,
#                          tr_complete_the_look, tr_specifications)).tocsr().astype('float32')
#     train_data = np.concatenate((train_data, train_features_in_order))

    validation_data = hstack((validation_features_in_order,vl_ohe_category_0, vl_ohe_category_1, vl_ohe_brand, vl_ohe_dominant_material,
                         vl_ohe_dominant_color, vl_ohe_actual_color, vl_ohe_product_type, vl_trans, vl_ohe_gender,
                         vl_ohe_care_instruction_0, vl_title, vl_product_details, vl_ohe_care_instruction_1,
                         vl_complete_the_look, vl_specifications)).tocsr().astype('float32')

    validation_data = np.concatenate((validation_features_in_order, validation_data))

#     test_data = hstack((test_features_in_order, te_ohe_category_0, te_ohe_category_1, te_ohe_brand, te_ohe_dominant_material,
#                         te_ohe_dominant_color, te_ohe_actual_color, te_ohe_product_type, te_trans, te_ohe_gender,
#                         te_ohe_care_instruction_0, te_title, te_product_details, te_ohe_care_instruction_1,
#                         te_complete_the_look, te_specifications)).tocsr().astype('float32')
#     test_data = np.concatenate((test_features_in_order, validation_data))

#     return train_data, validation_data, test_data
    return validation_data

In [133]:
image_features_path = os.path.abspath(current_folder + '/..') + '/Data/total_features.npz'
data_path = os.path.abspath(current_folder+ '/..') + '/Data'
# images_path = "/ceph/csedu-scratch/project/akaradathodi/Images/"
images_path = os.path.abspath(data_path + '/../../') + '/Images/'

In [135]:
test_df = pd.read_csv(data_path + "/test_records.csv")
validation_df = pd.read_csv(data_path + "/validation_records.csv")
train_df = pd.read_csv(data_path + "/train_records.csv")

In [146]:
test_data = generate_encodings_for_features(test_df, validation_df, train_df)

KeyError: '5a602b99699215068eed4f79aa1e5c98'